Project 2
Michael Bell
Rob Chesser
Adam Durar

In [14]:
import pandas as pd
import numpy as np
import os
import glob
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
#pip install sqlalchemy-utils for the above

Get list of CSV's from each data set

In [15]:
path = './quandl_csv_2019'
all_files = glob.glob(os.path.join(path, "*.csv"))

li = []

for filename in all_files:
    fn, ext = os.path.splitext(os.path.basename(filename))
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df[['Date','Adj_Open','Adj_Close']]
    df = df.rename(columns={"Date": "date", "Adj_Open": "adj_open", "Adj_Close": "adj_close"})
    df['symbol'] = fn
    li.append(df)
li

[             date    adj_open   adj_close symbol
 0      2020-01-14  128.620000  128.800000    AXP
 1      2020-01-13  127.840000  128.540000    AXP
 2      2020-01-10  127.580000  127.280000    AXP
 3      2020-01-09  126.490000  127.810000    AXP
 4      2020-01-08  124.050000  125.540000    AXP
 ...           ...         ...         ...    ...
 12005  1972-06-07    1.855899    1.855899    AXP
 12006  1972-06-06    1.878225    1.878225    AXP
 12007  1972-06-05    1.984539    1.984539    AXP
 12008  1972-06-02    2.024230    2.024230    AXP
 12009  1972-06-01    2.033444    2.033444    AXP
 
 [12010 rows x 4 columns],              date    adj_open   adj_close symbol
 0      2020-01-14  147.450000  146.680000    CAT
 1      2020-01-13  146.530000  146.820000    CAT
 2      2020-01-10  147.450000  146.130000    CAT
 3      2020-01-09  147.810000  147.310000    CAT
 4      2020-01-08  146.830000  147.680000    CAT
 ...           ...         ...         ...    ...
 14604  1962-01-08    

Flatten DF Lists into DF

In [16]:
djdata = pd.concat(li, axis=0, ignore_index=True)
djdata['date'] = djdata['date'].astype('datetime64[ns]') #convert to date
djdata.head()

,date,adj_open,adj_close,symbol
0,2020-01-14,128.62,128.80,AXP
1,2020-01-13,127.84,128.54,AXP
2,2020-01-10,127.58,127.28,AXP
3,2020-01-09,126.49,127.81,AXP
4,2020-01-08,124.05,125.54,AXP


Print Row/Col counts

In [17]:
print(djdata.shape)

(208070, 4)


Connect to database or create if it doesn't exist

In [19]:
connection_string = "postgres:1217Eer4$@localhost:5432/tbm"
engine = create_engine(f'postgresql://{connection_string}')
if not database_exists(engine.url):  #pip install sqlalchemy-utils
    create_database(engine.url)

Drop table if it exists

In [20]:
engine.execute('DROP TABLE if exists "stock_data_2019"')

Check to be sure the table is dropped

In [21]:
engine.table_names()

['stock_data']

Create fresh table

In [22]:
from sqlalchemy import Table, Column, Integer, String, MetaData, Float, Date, ForeignKey

metadata = MetaData()
sdata = Table('stock_data', metadata,
    Column('id', Integer, primary_key=True),
    Column('date', Date),
    Column('adj_open', Float),
    Column('adj_close', Float),
    Column('symbol', String),
    Column('end_val', Float),
)

metadata.create_all(engine)


Check if table is created

In [23]:
engine.table_names()

['stock_data']

Set the index to the id field and reset the index count after concat

In [24]:
djdata.index.name = 'id'
djdata.reset_index()
djdata.head()
djdata.columns

Index(['date', 'adj_open', 'adj_close', 'symbol'], dtype='object')

In [25]:
djdata2 = pd.DataFrame()
djdata2 = djdata

yrs = []
syms = []
for yr in djdata2['date'].dt.year:
    if yr not in yrs and yr >= 2000:
        yrs.append(yr) 
#print(yrs)
for sym in djdata2['symbol']:
    if sym not in syms:
        syms.append(sym)

li2 = []
for sym in syms:
    ticker_all = djdata2[djdata['symbol'] == sym]
    for yr in yrs:
        ticker_yr = ticker_all[ticker_all['date'].dt.year == yr] 
        min_dt = min(ticker_yr['date'])
        max_dt = max(ticker_yr['date'])
        ticker_yr['date'].dtypes
        ticker_min = ticker_yr[ticker_yr['date'] == min_dt.date()]
        ticker_max = ticker_yr[ticker_yr['date'] == max_dt.date()]
        ao_min = list(ticker_min['adj_open'])[0]
        ac_min = list(ticker_min['adj_close'])[0]
        ao_max = list(ticker_max['adj_open'])[0]
        ac_max = list(ticker_max['adj_close'])[0]
        #((Close: Last Trading Day of the Year) - (Open: First Trading Day of the Year))/(Open: First Trading Day of the Year)
        ticker_min['end_val'] = ((ac_max-ao_min)/ao_min)
        ticker_max['end_val'] = ((ac_max-ao_min)/ao_min)
        df_min = pd.DataFrame(ticker_min, columns =['date', 'adj_open', 'adj_close', 'symbol','end_val'])
        df_max = pd.DataFrame(ticker_max, columns =['date', 'adj_open', 'adj_close', 'symbol','end_val'])
        li2.append(df_min)
        li2.append(df_max)

df_all = pd.concat(li2, axis=0, ignore_index=True)

df_all.index.name = 'id'
df_all.reset_index()
        
df_all.head()
        
        

C:\Users\Michael Bell\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
C:\Users\Michael Bell\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
C:\Users\Michael Bell\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,date,adj_open,adj_close,symbol,end_val
id,,,,,
0,2020-01-02,124.660000,125.850000,AXP,0.033210
1,2020-01-14,128.620000,128.800000,AXP,0.033210
2,2019-01-02,92.232993,93.971385,AXP,0.345138
3,2019-12-31,123.866776,124.066095,AXP,0.345138
4,2018-01-02,96.555237,95.790385,AXP,-0.030422


Insert data into Postgres

In [26]:
df_all.to_sql(name='stock_data_2019', con=engine, if_exists='append', index=True)

Show sample data

In [27]:
engine.execute('SELECT * FROM stock_data_2019 LIMIT 5').fetchall()

[(0, datetime.datetime(2020, 1, 2, 0, 0), 124.66, 125.85, 'AXP', 0.0332103321033212),
 (1, datetime.datetime(2020, 1, 14, 0, 0), 128.62, 128.8, 'AXP', 0.0332103321033212),
 (2, datetime.datetime(2019, 1, 2, 0, 0), 92.2329927679586, 93.9713848156563, 'AXP', 0.34513791065449),
 (3, datetime.datetime(2019, 12, 31, 0, 0), 123.866776211593, 124.066095185302, 'AXP', 0.34513791065449),
 (4, datetime.datetime(2018, 1, 2, 0, 0), 96.5552365653005, 95.7903850974715, 'AXP', -0.0304222030629708)]

Check row counts

In [28]:
engine.execute('SELECT count(*) FROM stock_data_2019').fetchall()

[(714,)]